<a href="https://colab.research.google.com/github/liviarainho/Caixeiro-Viajante/blob/main/Trabalho_Otimiza%C3%A7%C3%A3o_de_Rota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pip Installs

In [ ]:
!pip install ortools
!pip install deap
!pip install googlemaps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 885.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 925.4 kB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.1 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have prot

# Imports

In [ ]:
import json
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.distance import geodesic
from ortools.constraint_solver import routing_enums_pb2, pywrapcp
import folium
import networkx as nx

import random
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt
from IPython.display import display
import warnings
import googlemaps
from scipy.spatial import distance_matrix

# Organização do Dataset

In [ ]:
# Carregar os dados das cidades
filepath = "/content/geojs-35-mun.json"
with open(filepath, "r", encoding='utf-8') as file:
    data = json.load(file)
features = data['features']
cities = pd.DataFrame([
    {"City": feature['properties']['name'],
     "Latitude": feature['geometry']['coordinates'][0][0][1],
     "Longitude": feature['geometry']['coordinates'][0][0][0]}
    for feature in features
])

# Calcular a matriz de distâncias usando a fórmula de Haversine
def haversine(coord1, coord2):
    R = 6371  # Raio da Terra em km
    lat1, lon1 = np.radians(coord1)
    lat2, lon2 = np.radians(coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return int(distance)

distance_matrix = []
for i, coord1 in enumerate(cities[['Latitude', 'Longitude']].values):
    distance_matrix.append([])
    for coord2 in cities[['Latitude', 'Longitude']].values:
        distance_matrix[-1].append(haversine(coord1, coord2))

# OR-Tools

In [ ]:
# OR-Tools setup
D = distance_matrix

# Definindo 'São Paulo' como nó inicial e final da rota
start_node = list(cities['City']).index('São Paulo')
manager = pywrapcp.RoutingIndexManager(len(D), 1, start_node)
routing = pywrapcp.RoutingModel(manager)

# Callback de distância
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return D[from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

solution = routing.SolveWithParameters(search_parameters)

# Função para imprimir solução
def print_solution(manager, routing, solution):
    print(f'Menor distância para rota: {solution.ObjectiveValue()} km.')
    index = routing.Start(0)
    plan_output = 'Rota:\n'
    while not routing.IsEnd(index):
        plan_output += f' {cities.iloc[manager.IndexToNode(index)]["City"]} ->'
        index = solution.Value(routing.NextVar(index))
    plan_output += f' {cities.iloc[manager.IndexToNode(index)]["City"]}\n'
    print(plan_output)

if solution:
    print_solution(manager, routing, solution)

# Função para obter rotas
def get_routes(solution, routing, manager):
    route = []
    index = routing.Start(0)
    while not routing.IsEnd(index):
        route.append(manager.IndexToNode(index))
        index = solution.Value(routing.NextVar(index))
    return route

routes = get_routes(solution, routing, manager)

# Obter nomes das cidades na rota
route_cities = [cities.iloc[i]["City"] for i in routes]

print(route_cities)

# Criar o mapa da rota
map_ortools = folium.Map(location=[-23.5489, -46.6388], zoom_start=8)
for i in range(len(routes) - 1):
    start_city = cities.iloc[routes[i]]
    end_city = cities.iloc[routes[i + 1]]
    folium.PolyLine(
        locations=[
            [start_city['Latitude'], start_city['Longitude']],
            [end_city['Latitude'], end_city['Longitude']]
        ],
        color='blue'
    ).add_to(map_ortools)
display(map_ortools)

Menor distância para rota: 10279 km.
Rota:
 São Paulo -> Mairiporã -> Caieiras -> Osasco -> Carapicuíba -> Barueri -> Jandira -> Cotia -> Vargem Grande Paulista -> Itapevi -> São Roque -> Araçariguama -> Cabreúva -> Pirapora do Bom Jesus -> Santana de Parnaíba -> Cajamar -> Franco da Rocha -> Francisco Morato -> Campo Limpo Paulista -> Várzea Paulista -> Jundiaí -> Louveira -> Vinhedo -> Itupeva -> Indaiatuba -> Salto -> Itu -> Sorocaba -> Mairinque -> Alumínio -> Votorantim -> Ibiúna -> Piedade -> Salto de Pirapora -> Araçoiaba da Serra -> Iperó -> Capela do Alto -> Alambari -> Sarapuí -> Pilar do Sul -> Tapiraí -> Juquiá -> Sete Barras -> Registro -> Iguape -> Itariri -> Miracatu -> Pedro de Toledo -> Juquitiba -> São Lourenço da Serra -> Itanhaém -> Embu-Guaçu -> Itapecerica da Serra -> Embu -> Taboão da Serra -> Diadema -> São Caetano do Sul -> Santo André -> São Bernardo do Campo -> São Vicente -> Praia Grande -> Mongaguá -> Peruíbe -> Ilha Comprida -> Pariquera-Açu -> Jacupiranga

# Algoritmo Guloso

In [ ]:
# Resolver TSP usando Algoritmo Guloso
G = nx.Graph()
for i in range(len(distance_matrix)):
    for j in range(i + 1, len(distance_matrix)):
        G.add_edge(i, j, weight=distance_matrix[i][j])

tsp_path = nx.approximation.traveling_salesman_problem(G, cycle=True, method=nx.approximation.greedy_tsp)
total_distance_greedy = sum(distance_matrix[tsp_path[i]][tsp_path[i + 1]] for i in range(len(tsp_path) - 1))

print(f"Greedy Algorithm: Total Distance = {total_distance_greedy} km")

# Criar o mapa da rota
map_greedy = folium.Map(location=[-23.5489, -46.6388], zoom_start=8)
for i in range(len(tsp_path) - 1):
    start_city = cities.iloc[tsp_path[i]]
    end_city = cities.iloc[tsp_path[i + 1]]
    folium.PolyLine(
        locations=[
            [start_city['Latitude'], start_city['Longitude']],
            [end_city['Latitude'], end_city['Longitude']]
        ],
        color='blue'
    ).add_to(map_greedy)
display(map_greedy)

Greedy Algorithm: Total Distance = 12029 km


## Algoritmo Genético

In [ ]:
# Resolver TSP usando Algoritmo Genético
def eval_tsp(individual):
    total_distance = 0
    for i in range(len(individual) - 1):
        total_distance += distance_matrix[individual[i]][individual[i + 1]]
    total_distance += distance_matrix[individual[-1]][individual[0]]
    return (total_distance,)

def create_individual(nodes):
    nodes = nodes.copy()
    nodes.remove(0)
    individual = [0] + random.sample(nodes, len(nodes))
    return individual

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("indices", create_individual, list(range(len(distance_matrix))))
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", eval_tsp)

population = toolbox.population(n=300)
hof = tools.HallOfFame(1)

algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=5000, halloffame=hof, verbose=False)

best_individual = hof[0]
total_distance_genetic = eval_tsp(best_individual)[0]

print(f"Genetic Algorithm: Total Distance = {total_distance_genetic} km")

# Criar o mapa da rota
map_genetic = folium.Map(location=[-23.5489, -46.6388], zoom_start=8)
for i in range(len(best_individual) - 1):
    start_city = cities.iloc[best_individual[i]]
    end_city = cities.iloc[best_individual[i + 1]]
    folium.PolyLine(
        locations=[
            [start_city['Latitude'], start_city['Longitude']],
            [end_city['Latitude'], end_city['Longitude']]
        ],
        color='red'
    ).add_to(map_genetic)
display(map_genetic)


Genetic Algorithm: Total Distance = 32417 km


# Algoritmo de Christofides

In [ ]:
# Carregar os dados do arquivo JSON
with open("geojs-35-mun.json", 'r', encoding='utf-8') as file:
    data = json.load(file)

# Criando um DataFrame com as coordenadas
features = data['features']
cities = pd.DataFrame([
    {"City": feature['properties']['name'],
     "Latitude": feature['geometry']['coordinates'][0][0][1],
     "Longitude": feature['geometry']['coordinates'][0][0][0]}
    for feature in features if len(feature['geometry']['coordinates'][0][0]) == 2
])

# Calculando a matriz de distâncias
coords = cities[["Latitude", "Longitude"]].values
dist_matrix = distance_matrix(coords, coords)

# Criando um grafo completo
G = nx.complete_graph(len(cities))
for i in range(len(cities)):
    for j in range(i + 1, len(cities)):
        G.add_edge(i, j, weight=dist_matrix[i][j])

# Algoritmo de Christofides para o problema do caixeiro viajante
tsp_route = nx.approximation.traveling_salesman_problem(G, weight="weight", cycle=True)

# Começar e terminar em São Paulo
sp_index = cities.index[cities["City"] == "São Paulo"][0]
if tsp_route[0] != sp_index:
    tsp_route = tsp_route[tsp_route.index(sp_index):] + tsp_route[:tsp_route.index(sp_index)]
tsp_route.append(tsp_route[0])

# Visualização no Folium
map_christofides = folium.Map(location=[-23.5489, -46.6388], zoom_start=8)

# Desenhando linhas entre os pontos
for i in range(1, len(tsp_route)):
    folium.PolyLine(
        locations=[
            [cities.iloc[tsp_route[i-1]]["Latitude"], cities.iloc[tsp_route[i-1]]["Longitude"]],
            [cities.iloc[tsp_route[i]]["Latitude"], cities.iloc[tsp_route[i]]["Longitude"]],
        ],
        color='blue'
    ).add_to(map_christofides)

# Exibir o mapa
map_christofides.save("tsp_route.html")

# Distância total
total_distance = sum(dist_matrix[tsp_route[i], tsp_route[i + 1]] for i in range(len(tsp_route) - 1))
total_distance += dist_matrix[tsp_route[-1], tsp_route[0]]  # Adiciona a distância para voltar ao ponto inicial

# Convertendo as distâncias de graus para quilômetros
total_kilometers = total_distance * 111

print(f"Algoritmo de Christofides: Distância Total = {total_kilometers:.2f} km")
map_christofides


Algoritmo de Christofides: Distância Total = 11713.92 km


# Algoritmo Tabu

In [ ]:
features = data['features']
cities = pd.DataFrame([
    {"City": feature['properties']['name'],
     "Latitude": feature['geometry']['coordinates'][0][0][1],
     "Longitude": feature['geometry']['coordinates'][0][0][0]}
    for feature in features if len(feature['geometry']['coordinates'][0][0]) == 2
])

# Matriz de distância
distance_matrix = []
for coord1 in cities[['Latitude', 'Longitude']].values:
    distance_matrix.append([haversine(coord1, coord2) for coord2 in cities[['Latitude', 'Longitude']].values])

# Busca Tabu
def initial_solution(num_cities):
    route = list(range(num_cities))
    random.shuffle(route)
    return route

def calculate_total_distance(route):
    return sum(distance_matrix[route[i]][route[i+1]] for i in range(-1, len(route) - 1))

def two_opt_swap(route):
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 2, len(route)):
                if j - i == 1: continue  # Eles são vizinhos consecutivos
                if distance_matrix[route[i - 1]][route[i]] + distance_matrix[route[j - 1]][route[j]] > \
                   distance_matrix[route[i - 1]][route[j - 1]] + distance_matrix[route[i]][route[j]]:
                    route[i:j] = route[j - 1:i - 1:-1]
                    improved = True
    return route

def tabu_search(initial_route, iterations, tabu_size, num_neighbors):
    best_route = initial_route[:]
    best_cost = calculate_total_distance(best_route)
    tabu_list = []

    current_route = best_route[:]
    current_cost = best_cost

    for _ in range(iterations):
        neighbors = []
        for _ in range(num_neighbors):
            new_route = current_route[:]
            l, r = sorted(random.sample(range(len(new_route)), 2))
            new_route[l:r+1] = reversed(new_route[l:r+1])
            neighbors.append(new_route)

        neighbors = sorted(neighbors, key=calculate_total_distance)

        for neighbor in neighbors:
            if neighbor not in tabu_list:
                neighbor_cost = calculate_total_distance(neighbor)
                if neighbor_cost < current_cost:
                    current_route = neighbor[:]
                    current_cost = neighbor_cost
                    if neighbor_cost < best_cost:
                        best_route = neighbor[:]
                        best_cost = neighbor_cost
                    break
        if len(tabu_list) >= tabu_size:
            tabu_list.pop(0)
        tabu_list.append(current_route)

    # Aplica o 2-opt para tentar remover cruzamentos
    best_route = two_opt_swap(best_route)
    best_cost = calculate_total_distance(best_route)
    return best_route, best_cost

# Execução da Busca Tabu
initial_route = initial_solution(len(cities))
best_route, best_distance = tabu_search(initial_route, 1000, 50, 50)

# Visualização
if best_route:
    map_tabu = folium.Map(location=[-23.5489, -46.6388], zoom_start=8)
    for i in range(len(best_route) - 1):
        start_city = cities.iloc[best_route[i]]
        end_city = cities.iloc[best_route[i+1]]
        folium.PolyLine(
            locations=[
                [start_city['Latitude'], start_city['Longitude']],
                [end_city['Latitude'], end_city['Longitude']]
            ],
            color='blue'
        ).add_to(map_tabu)
    display(map_tabu)
print(f"Total distance: {best_distance} km")


Total distance: 10815 km
